In [1]:
import json
import pandas as pd
from PIL import Image, ImageDraw
import requests
from io import BytesIO
import pandas as pd
import uuid
import numpy as np
import cv2

In [2]:
pd.set_option('display.max_rows', 200)


In [3]:
class Page:
    def __init__(self,input_file,local=False):
        self.file = input_file
        self.local = local
        self.get_json()
        
    def get_json(self):
        if self.local :
            with open(self.file) as f :
                self.json = json.load(f)
        else :
            file_name = self.file['input']['name']
            r = requests.get('https://auth.anuvaad.org/download/' + file_name)
            self.json = r.json()
        
    def get_page(self,page_index):
        page_path = self.json['rsp']['outputs'][0]['page_info'][page_index]
        page_path = '/'.join(page_path.split('/')[1:])
        page_request = requests.get('https://auth.anuvaad.org/download/' + page_path)
        return page_request.content
    
#     def get_page_path(self,page_index):
#         return self.json['rsp']['outputs'][0]['page_info'][page_index]
        
        
        

In [4]:
def draw_box(page_image,compare_corrds, save=False):
    
    #image = Image.open(BytesIO(page_image))
    image = Image.open(page_image)
   
    draw   = ImageDraw.Draw(image)
    
    for region in compare_corrds :
        if region['ground'] != None :
            ground = region['ground']['boundingBox']['vertices']
            draw.rectangle(((ground[2]['x'],  ground[2]['y']), (ground[0]['x'],ground[0]['y'])), outline= 'blue',width=5)

        if region['input'] != None:
            inpu_t = region['input']['boundingBox']['vertices']
            draw.rectangle(((inpu_t[2]['x'],  inpu_t[2]['y']), (inpu_t[0]['x'],inpu_t[0]['y'])), outline= 'red',width=5)

    #save_filepath = os.path.join(os.path.dirname(filepath), 'processed_' + os.path.basename(filepath))
    if save:
        image.save(str(uuid.uuid1()) + '.png')
    
    return image



def draw_region(page_image,corrds,color='red',  save=False):
    
    #image = Image.open(BytesIO(page_image))
    image = Image.open(page_image)
   
    draw   = ImageDraw.Draw(image)
    
    for region in corrds :
       
            ground = region['boundingBox']['vertices']
            draw.rectangle(((ground[2]['x'],  ground[2]['y']), (ground[0]['x'],ground[0]['y'])), outline= color,width=5)

    if save:
        image.save(str(uuid.uuid1()) + '.png')
    
    return image



def find_erros(eval_data,input_data,iou_threshold):
  
    count_erros = []
    iou_error   = []
    for file_index in range(len(eval_data['rsp']['outputs'])) :
        
        for page_index in range(len(eval_data['rsp']['outputs'][file_index]['pages'])):
           

            print('evaluationg file {} page {}'.format(file_index,page_index))

            page = eval_data['rsp']['outputs'][file_index]['pages'][page_index]
            page_image = input_data['rsp']['outputs'][file_index]['page_info'][page_index]

            regions = {}
            try :
                counts = page['count'] 
                count_diff  = abs(counts['input'] - counts['ground'])
                #print('diff in boxes is {}'.format(count_diff))

               # if count_diff >= count_threshold :
                        #count_erros.append({'file_index' : file_index ,'page_index': page_index,'page':page,'path':page_image})
                avg_iou = pd.DataFrame(page['iou'])['iou'].mean()
                print('avrage page iou is {} \n'.format(pd.DataFrame(page['iou'])['iou'].mean()))
                #regions = {'below_iou_threshold':[],'avg_iou': avg_iou, 'count_diff':count_diff,'count_input':counts['input'],'count_ground':counts['ground'], 'file_index' : file_index ,'page_index': page_index, 'page':page['iou'],'path':page_image}
                regions = {'below_iou_threshold':[],'avg_iou': avg_iou,'count_input':counts['input'],'count_ground':counts['ground'], 'file_index' : file_index ,'page_index': page_index, 'page':page['iou'],'path':page_image}


                for reguion_iou in page['iou']:
                    if reguion_iou['iou'] <= iou_threshold:
                        regions['below_iou_threshold'].append(reguion_iou)

                regions['count_below_iou_threshold']= len(regions['below_iou_threshold'])
                #iou_error.append(regions)

            except :
                pass
            
            iou_error.append(regions)
            
    return iou_error
    
    
    

In [15]:
image_base_path = '/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/'

input_path = '/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/evaluator/upload/craft_38.json'
ground_path = '/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/evaluator/upload/vision_38.json'

eval_path = '/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/evaluator/upload/evaluate_38_word.json'

In [16]:
# image_base_path = '/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/'

# input_path = '/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/evaluator/upload/0-1607093029482609.json'
# ground_path = '/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/ocr/google/upload/0-16070828096677043.json'

# #words
# #eval_path = '/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/evaluator/upload/0-16071576929145546.json'

# #line
# eval_path = '/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/evaluator/upload/0-1607160946359452.json'

In [17]:
with open(eval_path) as f :
    eval_data = json.load(f)
 

In [18]:
with open(input_path) as f :
    input_data = json.load(f)


In [19]:
with open(ground_path) as f :
    ground_data= _data = json.load(f)




In [34]:
inputs = pd.DataFrame(input_data['rsp']['inputs'])
inputs['file'].apply(lambda x : x['name'])

0            20200804144132705.pdf
1                        20521.pdf
2                        20695.pdf
3                        35977.pdf
4                        37429.pdf
5        Screenshot 2020-10-12.pdf
6      assamese_1_singlecolumn.pdf
7       assamese_2_siglecoulun.pdf
8       bengali_1_singlecolumn.pdf
9             bengali_3_single.pdf
10      bengali_3_singlecolumn.pdf
11                     class_3.pdf
12                     dict_16.pdf
13                       empty.pdf
14       gujrati_1_multicolumn.pdf
15      gujrati_1_singlecolumn.pdf
16         hind_1_singlecolumn.pdf
17               hindi_1_multi.pdf
18              hindi_1_single.pdf
19      kannada_1_singlecolumn.pdf
20      kannada_2_singlecolumn.pdf
21    long_paragraph-converted.pdf
22     malayalam_1_multicolumn.pdf
23    malayalam_1_singlecolumn.pdf
24     manipuri_2_singlecolumn.pdf
25      marathi_1_singlecolumn.pdf
26                       ncert.pdf
27       nepali_1_singlecolumn.pdf
28       nepali_2_si

In [35]:
inputs = pd.DataFrame(ground_data['rsp']['inputs'])
inputs['file'].apply(lambda x : x['name'])

0            20200804144132705.pdf
1                        20521.pdf
2                        20695.pdf
3                        35977.pdf
4                        37429.pdf
5        Screenshot 2020-10-12.pdf
6      assamese_1_singlecolumn.pdf
7       assamese_2_siglecoulun.pdf
8       bengali_1_singlecolumn.pdf
9             bengali_3_single.pdf
10      bengali_3_singlecolumn.pdf
11                     class_3.pdf
12                     dict_16.pdf
13                       empty.pdf
14       gujrati_1_multicolumn.pdf
15      gujrati_1_singlecolumn.pdf
16         hind_1_singlecolumn.pdf
17               hindi_1_multi.pdf
18              hindi_1_single.pdf
19      kannada_1_singlecolumn.pdf
20      kannada_2_singlecolumn.pdf
21    long_paragraph-converted.pdf
22     malayalam_1_multicolumn.pdf
23    malayalam_1_singlecolumn.pdf
24     manipuri_2_singlecolumn.pdf
25      marathi_1_singlecolumn.pdf
26                       ncert.pdf
27       nepali_1_singlecolumn.pdf
28       nepali_2_si

In [21]:
iou_errors =  find_erros(eval_data,input_data,iou_threshold = 0.33)

evaluationg file 0 page 0
avrage page iou is 1.0 

evaluationg file 1 page 0
avrage page iou is 1.0 

evaluationg file 2 page 0
avrage page iou is 1.0 

evaluationg file 2 page 1
avrage page iou is 1.0 

evaluationg file 2 page 2
avrage page iou is 1.0 

evaluationg file 2 page 3
avrage page iou is 1.0 

evaluationg file 2 page 4
avrage page iou is 1.0 

evaluationg file 2 page 5
avrage page iou is 1.0 

evaluationg file 3 page 0
avrage page iou is 1.0 

evaluationg file 3 page 1
avrage page iou is 1.0 

evaluationg file 4 page 0
avrage page iou is 1.0 

evaluationg file 4 page 1
avrage page iou is 1.0 

evaluationg file 4 page 2
avrage page iou is 1.0 

evaluationg file 4 page 3
avrage page iou is 1.0 

evaluationg file 5 page 0
avrage page iou is 1.0 

evaluationg file 6 page 0
avrage page iou is 1.0 

evaluationg file 6 page 1
avrage page iou is 1.0 

evaluationg file 6 page 2
avrage page iou is 1.0 

evaluationg file 7 page 0
avrage page iou is 1.0 

evaluationg file 7 page 1
avrag

In [22]:
erro_df = pd.DataFrame(iou_errors)
erro_df.columns

Index(['below_iou_threshold', 'avg_iou', 'count_input', 'count_ground',
       'file_index', 'page_index', 'page', 'path',
       'count_below_iou_threshold'],
      dtype='object')

In [23]:
erro_df[['file_index', 'page_index','avg_iou','count_input', 'count_ground','count_below_iou_threshold']]

,file_index,page_index,avg_iou,count_input,count_ground,count_below_iou_threshold
0,0.0,0.0,1.000000,270.0,270.0,0.0
1,1.0,0.0,1.000000,429.0,429.0,0.0
2,2.0,0.0,1.000000,63.0,63.0,0.0
3,2.0,1.0,1.000000,54.0,54.0,0.0
4,2.0,2.0,1.000000,297.0,297.0,0.0
5,2.0,3.0,1.000000,353.0,353.0,0.0
6,2.0,4.0,1.000000,213.0,213.0,0.0
7,2.0,5.0,1.000000,245.0,245.0,0.0
8,3.0,0.0,1.000000,114.0,114.0,0.0
9,3.0,1.0,1.000000,180.0,180.0,0.0


In [24]:
#word level iou = 0.5222075391815083
#line level iou = 0.6396832369726599
erro_df['avg_iou'].mean()

0.9999882631863102

In [25]:
count_row = 0
page = erro_df.iloc[count_row]
#draw_box(image_base_path + page['path'],page['below_iou_threshold'])

In [26]:
page = erro_df.iloc[count_row]
draw_box(image_base_path + page['path'],page['page'])

FileNotFoundError: [Errno 2] No such file or directory: '/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/20200804144132705_a86b3e49-be3b-4739-bafd-213c1b8a260d/images/202008041441327050001-1.jpg'

In [32]:
import glob
def get_box(file_index):
    pages = len(input_data['rsp']['outputs'][file_index]['page_info'])
    for page_index in range(pages):
        #try:
        page_image = input_data['rsp']['outputs'][file_index]['page_info'][page_index]
        compare_corrds =   eval_data['rsp']['outputs'][file_index]['pages'][page_index]['iou']
        gt_image = draw_region(image_base_path + page_image,ground_data['rsp']['outputs'][file_index]['pages'][page_index]['words'])
        craft_image = draw_region(image_base_path + page_image,input_data['rsp']['outputs'][file_index]['pages'][page_index]['words'] ,color='blue')
        comapre_image = draw_box(image_base_path + page_image,compare_corrds)
        stiched_image = np.hstack([np.array(gt_image) ,np.array(craft_image)])
        stiched_image = np.hstack([np.array(gt_image) , np.array(comapre_image) ,np.array(craft_image)])
        cv2.imwrite("/home/naresh/word_compare4/"+page_image.split("/")[-1], stiched_image)
        #except:
            #pass

In [33]:
files = len(input_data['rsp']['outputs'])
for file_index in range(files):
    get_box(file_index)

FileNotFoundError: [Errno 2] No such file or directory: '/home/naresh/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload/20200804144132705_a86b3e49-be3b-4739-bafd-213c1b8a260d/images/202008041441327050001-1.jpg'

In [31]:
files

38

In [ ]:
file_index = 1
page_index = 0

page_image = input_data['rsp']['outputs'][file_index]['page_info'][page_index]
compare_corrds =   eval_data['rsp']['outputs'][file_index]['pages'][page_index]['iou']

gt_image = draw_region(image_base_path + page_image,ground_data['rsp']['outputs'][file_index]['pages'][page_index]['words'])
craft_image = draw_region(image_base_path + page_image,input_data['rsp']['outputs'][file_index]['pages'][page_index]['words'] ,color='blue')
comapre_image = draw_box(image_base_path + page_image,compare_corrds)
compare_df = pd.DataFrame(compare_corrds)
print(compare_df['iou'].mean())
#iou_stats.append(compare_df['iou'].mean())
stiched_image = np.hstack([np.array(gt_image) , np.array(comapre_image) ,np.array(craft_image)])
cv2.imwrite(  'results/' +str(uuid.uuid1()) + '.png', stiched_image)

In [ ]:
eval_data

In [ ]:
count_df = pd.DataFrame(iou_errors)
count_df

In [ ]:
iou_df = pd.DataFrame(iou_errors)
iou_df

In [ ]:
def draw_low_iou_threshold(iou_df):
    files = iou_df.file_index.unique()
    for file_index in files:
        file_df = iou_df[iou_df["file_index"]=file_index]
        file_df = iou_df[iou_df["file_index"]=file_index]
        for page_index in range(len(file_df)):
            page = file_df.iloc[page_index]
            draw_box(image_base_path + page['path'],page['below_iou_threshold'])
            

In [ ]:
draw_low_iou_threshold(iou_df)

In [ ]:
iou_row = 4
page = iou_df.iloc[iou_row]
draw_box(image_base_path + page['path'],page['page'])


In [ ]:
page

In [ ]:
iou_stats = []

for file_index in range(len(eval_data['rsp']['outputs'])) :
    for page_index in range(len(eval_data['rsp']['outputs'][file_index]['pages'])):
        page_image = input_data['rsp']['outputs'][file_index]['page_info'][page_index]
        compare_corrds =   eval_data['rsp']['outputs'][file_index]['pages'][page_index]['iou']
        
        gt_image = draw_region(image_base_path + page_image,ground_data['rsp']['outputs'][file_index]['pages'][page_index]['words'])
        craft_image = draw_region(image_base_path + page_image,input_data['rsp']['outputs'][file_index]['pages'][page_index]['words'] ,color='blue')
        comapre_image = draw_box(image_base_path + page_image,compare_corrds)
        compare_df = pd.DataFrame(compare_corrds)
        print(compare_df['iou'].mean())
        iou_stats.append(compare_df['iou'].mean())
        stiched_image = np.hstack([np.array(gt_image) , np.array(comapre_image) ,np.array(craft_image)])
        
        cv2.imwrite(str(uuid.uuid1()) + '.png', stiched_image)

In [ ]:
comapre_image

In [ ]:

compare_df = pd.DataFrame(compare_corrds)

In [ ]:
compare_df['iou'].mean()

In [ ]:
len(eval_data['rsp']['outputs']) #[file_index]

In [ ]:
eval_data['rsp'].keys()

In [ ]:
y =0.5849854404676739

2* y /( 1  +y)

In [ ]:
np.mean(iou_stats)

In [ ]:
np.mean(iou_stats)

In [ ]:
8,2,4

0.5331896355570389
0.5911757986732217
0.6203633723763122
0.5620582149400676
0.5488797296908341
0.535142327697838
0.6078407486631073
0.6884819741198223
0.5666216307191195
0.5940801542905321
0.6866600642338573
0.7034852042635412
0.5347262928269113
0.6035566594439823
0.5596672386558638
0.4062571461792611
0.5422557423269335
0.6512537552823257

0.5036370285005956
0.596914331076525
0.6246971532915176
0.5424381227153756
0.5575604059387527
0.5485543730261239
0.5943804339737555
0.6731481219035321
0.5769426378134691
0.5839788432017801
0.6676718033712223
0.6879190446933335
0.5731670231767895
0.5801987258267257
0.5901110920900879
0.38902491717860144
0.5584404078149451
0.6809534628249947
